In [35]:
import pandas as pd
from dbfread import DBF
from simpledbf import Dbf5
import duckdb
import polars as pl

In [112]:
def read_df(filename):
    dbf = DBF(filename, encoding="iso-8859-1")
    return pl.DataFrame(iter(dbf))


In [174]:
df = read_df(r"C:\Users\Murilo\Downloads\dbf\RDAC0801.dbf")

In [115]:

df2 = read_df(r"C:\Users\Murilo\Downloads\dbf\RDSP2301.dbf")

In [201]:
cols = set(['UF_ZI','ANO_CMPT','MES_CMPT','ESPEC','CGC_HOSP','N_AIH','IDENT','CEP','MUNIC_RES','NASC','SEXO','UTI_MES_IN','UTI_MES_AN','UTI_MES_AL','UTI_MES_TO','MARCA_UTI','UTI_INT_IN','UTI_INT_AN','UTI_INT_AL','UTI_INT_TO','DIAR_ACOM','QT_DIARIAS','PROC_SOLIC','PROC_REA','VAL_SH','VAL_SP','VAL_SADT','VAL_RN','VAL_ACOMP','VAL_ORTP','VAL_SANGUE','VAL_SADTSR','VAL_TRANSP','VAL_OBSANG','VAL_PED1AC','VAL_TOT','VAL_UTI','US_TOT','DI_INTER','DT_SAIDA','DIAG_PRINC','DIAG_SECUN','COBRANCA','NATUREZA','NAT_JUR','DESTAO','RUBRICA','IND_VDRL','MUNIC_MOV','COD_IDADE','IDADE','DIAS_PERM','MORTE','NACIONAL','NUM_PROC','CAR_INT','TOT_PT_SP','CPF_AUT','HOMONIMO','NUM_FILHOS','INSTRU','CID_NOTIF','CONTRACEP1','CONTRACEP2','GESTRISCO','INSC_PN','SEQ_AIH5','CBOR','CNAER','VINCPREV','GESTOR_COD','GESTOR_TP','GESTOR_CPF','GESTOR_DT','CNES','CNPJ_MANT','INFEHOSP','CID_ASSO','CID_MORTE','COMPLEX','FINANC','FAEC_TP','REGCT','RACA_COR','ETNIA','SEQUENCIA','REMESSA','AUD_JUST','SIS_JUST','VAL_SH_FED','VAL_SP_FED','VAL_SH_GES','VAL_SP_GES','VAL_UCI','MARCA_UCI','DIAGSEC1','DIAGSEC2','DIAGSEC3','DIAGSEC4','DIAGSEC5','DIAGSEC6','DIAGSEC7','DIAGSEC8','DIAGSEC9','TPDISEC1','TPDISEC2','TPDISEC3','TPDISEC4','TPDISEC5','TPDISEC6','TPDISEC7','TPDISEC8','TPDISEC9' ])
set(cols) - set(df2.columns)

# filtered_df = df.filter()
# filtered_df
value = "120030"
for column in df.columns:
    series = df[column]

    result = series.cast(pl.Utf8).str.contains(value).sum()
    if result > 0:
        print(column)
        pass
# filterd_df = df.with_columns(pl.col("*").is_in("1210106762").alias("has_value"))
# filterd_df


# df.filter(pl.col("INSC_PN").eq("000000000000000"))

df["COMPLEX"]


MUNIC_RES
MUNIC_MOV


COMPLEX
str
"""02"""
"""02"""
"""02"""
"""02"""
"""02"""
"""02"""
"""02"""
"""02"""
"""02"""


In [158]:
def upsert_column(df: pl.DataFrame, name: str, dtype: pl.PolarsDataType):
    if name in df.columns:
        return pl.col(name).cast(dtype)
    else:
        return pl.lit(None, dtype).alias(name)


def map_sih_rd(df: pl.DataFrame):
    return df.with_columns(
        pl.when(pl.col(pl.Utf8).str.len_chars() == 0)
        .then(None)
        .otherwise(pl.col(pl.Utf8))
        .name.keep(),
        pl.when(pl.col("IDADE").gt(200))
        .then(None)
        .otherwise(pl.col("IDADE"))
        .name.keep(),
    ).with_columns(
        pl.when(pl.col("GESTOR_CPF").cast(pl.UInt64) == 0)
        .then(None)
        .otherwise(pl.col("GESTOR_CPF").str.lstrip("0"))
        .name.keep(),
        pl.when(pl.col("INSC_PN").cast(pl.UInt64) == 0)
        .then(None)
        .otherwise(pl.col("INSC_PN").str.lstrip("0"))
        .name.keep(),
    ).with_columns(
        [
            pl.col("UF_ZI").cast(pl.UInt32),
            pl.col("ANO_CMPT").cast(pl.UInt16),
            pl.col("MES_CMPT").cast(pl.UInt8),
            pl.col("ESPEC").cast(pl.UInt8),
            upsert_column(df, "CGC_HOSP", pl.Utf8),
            upsert_column(df, "N_AIH", pl.Utf8),
            pl.col("IDENT").cast(pl.UInt8),
            upsert_column(df, "CEP", pl.Utf8),
            pl.col("MUNIC_RES").cast(pl.UInt32),
            pl.col("NASC").str.to_date("%Y%m%d"),
            pl.col("SEXO").cast(pl.UInt8),
            pl.col("UTI_MES_IN").cast(pl.UInt8),
            pl.col("UTI_MES_AN").cast(pl.UInt8),
            pl.col("UTI_MES_AL").cast(pl.UInt8),
            pl.col("UTI_MES_TO").cast(pl.UInt16),
            pl.col("MARCA_UTI").cast(pl.UInt8),
            pl.col("UTI_INT_IN").cast(pl.UInt8),
            pl.col("UTI_INT_AN").cast(pl.UInt8),
            pl.col("UTI_INT_AL").cast(pl.UInt8),
            pl.col("UTI_INT_TO").cast(pl.UInt16),
            pl.col("DIAR_ACOM").cast(pl.UInt16),
            pl.col("QT_DIARIAS").cast(pl.UInt16),
            pl.col("DT_INTER").str.to_date("%Y%m%d"),
            pl.col("DT_SAIDA").str.to_date("%Y%m%d"),
            upsert_column(df, "NAT_JUR", pl.Utf8),
            pl.col("IDADE").cast(pl.UInt8),
            pl.col("DIAS_PERM").cast(pl.UInt16),
            pl.col("MORTE").cast(pl.Boolean),
            pl.col("NACIONAL").cast(pl.UInt16),
            pl.col("CAR_INT").cast(pl.UInt8),
            pl.col("TOT_PT_SP").cast(pl.UInt32),
            pl.col("HOMONIMO").cast(pl.UInt8),
            pl.col("NUM_FILHOS").cast(pl.UInt8),
            pl.col("INSTRU").cast(pl.UInt8),
            pl.col("CONTRACEP1").cast(pl.UInt8),
            pl.col("CONTRACEP2").cast(pl.UInt8),
            pl.col("GESTRISCO").cast(pl.UInt8),
            pl.col("INSC_PN").cast(pl.Utf8),
            pl.col("SEQ_AIH5").cast(pl.Utf8),
            pl.col("CBOR").cast(pl.UInt16),
            pl.col("CNAER").cast(pl.UInt16),
            pl.col("VINCPREV").cast(pl.UInt8),
            pl.col("GESTOR_COD").cast(pl.UInt16),
            pl.col("GESTOR_TP").cast(pl.UInt8),
            pl.col("GESTOR_CPF").cast(pl.Utf8),
            pl.col("GESTOR_DT").str.to_date("%Y%m%d"),
            pl.col("INFEHOSP").cast(pl.UInt8),
            pl.col("FINANC").cast(pl.UInt8),
            pl.col("FAEC_TP").cast(pl.UInt32),
            pl.col("REGCT").cast(pl.UInt16),
            pl.col("RACA_COR").cast(pl.UInt16),
            pl.col("ETNIA").cast(pl.UInt16),
            pl.col("SEQUENCIA").cast(pl.UInt64),
            upsert_column(df, "AUD_JUST", pl.Utf8),
            upsert_column(df, "SIS_JUST", pl.Utf8),
            upsert_column(df, "VAL_SH_FED", pl.Float64),
            upsert_column(df, "VAL_SP_FED", pl.Float64),
            upsert_column(df, "VAL_SH_GES", pl.Float64),
            upsert_column(df, "VAL_SP_GES", pl.Float64),
            upsert_column(df, "VAL_UCI", pl.Float64),
            upsert_column(df, "MARCA_UCI", pl.UInt8),
            upsert_column(df, "DIAGSEC1", pl.Utf8),
            upsert_column(df, "DIAGSEC2", pl.Utf8),
            upsert_column(df, "DIAGSEC3", pl.Utf8),
            upsert_column(df, "DIAGSEC4", pl.Utf8),
            upsert_column(df, "DIAGSEC5", pl.Utf8),
            upsert_column(df, "DIAGSEC6", pl.Utf8),
            upsert_column(df, "DIAGSEC7", pl.Utf8),
            upsert_column(df, "DIAGSEC8", pl.Utf8),
            upsert_column(df, "DIAGSEC9", pl.Utf8),
            upsert_column(df, "TPDISEC1", pl.UInt8),
            upsert_column(df, "TPDISEC2", pl.UInt8),
            upsert_column(df, "TPDISEC3", pl.UInt8),
            upsert_column(df, "TPDISEC4", pl.UInt8),
            upsert_column(df, "TPDISEC5", pl.UInt8),
            upsert_column(df, "TPDISEC6", pl.UInt8),
            upsert_column(df, "TPDISEC7", pl.UInt8),
            upsert_column(df, "TPDISEC8", pl.UInt8),
            upsert_column(df, "TPDISEC9", pl.UInt8),
        ]
    )


map_sih_rd(df)["INSC_PN"]


C:\Users\Murilo\AppData\Local\Temp\ipykernel_17328\1617002060.py:21: DeprecationWarning: `lstrip` is deprecated. It has been renamed to `strip_chars_start`.
  .otherwise(pl.col("GESTOR_CPF").str.lstrip("0"))
C:\Users\Murilo\AppData\Local\Temp\ipykernel_17328\1617002060.py:25: DeprecationWarning: `lstrip` is deprecated. It has been renamed to `strip_chars_start`.
  .otherwise(pl.col("INSC_PN").str.lstrip("0"))


INSC_PN
str
null
null
null
null
null
null
null
null
null


In [28]:
s1 = pl.Series([1, 2, 3, 4, 5], dtype=pl.Int16)
s2 = pl.Series(["Murilo", "Matheus", "Graca", "Milton", "Mujica"], dtype=pl.Utf8)
s3 = pl.Series([True, True, True, False, False], dtype=pl.Boolean)
s4 = pl.Series(["10/10/2022", "10/10/2022", "10/10/2022", "05/10/2022", "07/05/2023"], dtype=pl.Utf8)
test_df = pl.DataFrame({
    "id": s1,
    "name": s2,
    "boolean": s3,
    "dates": s4
})

test_df = test_df.with_columns([
    pl.col("dates").str.to_date()
])

test_df

id,name,boolean,dates
i16,str,bool,date
1,"""Murilo""",true,2022-10-10
2,"""Matheus""",true,2022-10-10
3,"""Graca""",true,2022-10-10
4,"""Milton""",false,2022-10-05
5,"""Mujica""",false,2023-05-07


In [173]:
with duckdb.connect(r"C:\Users\Murilo\Downloads\duckdb\test.db") as con:
    # result = con.query("SELECT * FROM SIH_RD_SISTEMA_INFORMACOES_HOSPITALARES")
    result = con.sql("DROP table IF EXISTS SIH_RD_SISTEMA_INFORMACOES_HOSPITALARES")
    result = con.sql("""delete from main."__import" where table_name = 'SIH_RD_SISTEMA_INFORMACOES_HOSPITALARES'""")
    # result = con.execute(f"SELECT file FROM __import where table_name = ?", ["blabla"]).df()
    print(result)



CatalogException: Catalog Error: Table with name SIH_RD_SISTEMA_INFORMACOES_HOSPITALARES does not exist!
Did you mean "sqlite_master"?